In [4]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

## Use decision tree as a rough way for splitting based on that feature
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import tree

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

In [17]:
(20*13)/60

4.333333333333333

In [16]:
#Now lets adapt this for all regions
# Add the following lines
from constants import regions

#Loop everything 
#AND add region into the filename

for region in regions:
    pass

In [13]:
regions

['Prefrontal',
 'Frontal',
 'Left Frontal',
 'Right Frontal',
 'Central',
 'Left Central',
 'Right Central',
 'Left Temporal',
 'Right Temporal',
 'Parietal',
 'Left Parietal',
 'Right Parietal',
 'Occipital']

In [5]:
sys.argv[1] = 2
sys.argv[2] = 2

In [6]:
# 1 Data type is defined by script input ______________________________________________________
d_num = int( sys.argv[1] )
data_type = ['N2', 'N3','REM', 'Wake', 'N1'][d_num]

expt_num = int( sys.argv[2] )

#2. Load the data based on data type ___________________________________________________________
df = joblib.load(os.path.join(os.path.abspath('..'), data_type + '_psd_normalised_data.pkl') )

single_region_df = df[[col for col in df.columns if col.endswith('_Central')]]
frequency_vals  = np.arange(0.5,40.125, 0.125)
single_region_df.columns = frequency_vals

folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
paths = joblib.load(folder + data_type + '_paths.pkl')

groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3})

In [7]:
#3 Generate the frequency band search space ____________________________________________________
width_slide_list = [(0.5,0.5)] 
for window_width in range(1, 15 ):
    width_slide_list.append((window_width, 0.5) )

In [8]:
#4 Calculate all of the features _____________________________

feature_dfs = []

for window_width_hz, window_slide_hz in width_slide_list:
    description = 'window width : ' + str(window_width_hz) + ' .  window stride : ' + str(window_slide_hz)
    
    #1.Set Window width in Hz and Window Slide in Hz
    # window_width_hz = 3
    window_len = (window_width_hz/0.125)+1
    assert window_len%1 == 0
    window_len = int(window_len)
    
    # window_slide_hz = 0.5 
    window_slide_len = window_slide_hz/0.125 
    assert window_slide_len%1 == 0
    window_slide_len = int(window_slide_len)
    
    
    #2.Calculate band values and store in dataframe calculated_df
    
    # Start of the window is the middle_freq value minus window_width_hz/2 
    # End of the window is the middle freq value plus window_width_hz/2
    window_len
    
    middle_freq = []
    final_cols = []
    
    i = 0
    while i < len(frequency_vals) - window_len:
        middle_freq.append( frequency_vals[i:i+window_len].mean() )
        band_vals = single_region_df.iloc[:,i:i+window_len].mean(axis = 1) #For every row calculate the mean for the appropriate elements
        
        final_cols.append(band_vals)
        
        # scaled_psd.append( psd_values[i:i+factor].mean() )
        i+= window_slide_len
    
    calculated_df = pd.DataFrame(final_cols).T
    calculated_df.columns = middle_freq
    
    calculated_df.columns = [ str(col) + '_width_' + str(window_width_hz) for col in calculated_df.columns]
    
    
    #3.Calculate Information Gain Based on Features
    #Change to binary HC vs PD / PD+RBD ---> Generate expt specific X,y,groups 
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(calculated_df,y,groups,expt_num)

    feature_dfs.append(X_expt)
        
#5 Concatenate all feature_dfs into a single dataframe_________________________________________________________________
X_expt_concatenated = pd.concat(feature_dfs, axis=1)

cols_list = list(X_expt_concatenated.columns)

col_combos = []
for col_1 in cols_list:
    ind = cols_list.index(col_1)
    for col_2 in cols_list[ind+1:]:
        col_combos.append([ col_1 , col_2 ] )

scores = []
for col_combo in col_combos: 
    X_selected = X_expt_concatenated[ col_combo ]
    # clf = RidgeClassifier().fit(X_selected, y_expt)
    clf = LinearDiscriminantAnalysis().fit(X_selected, y_expt)
    score = accuracy_score(y_expt,  clf.predict(X_selected))
    scores.append(score)

f_name =  'Results/' + data_type + '_expt_' + str(expt_num) + '_scores.pkl'
joblib.dump(scores,f_name)

f_name =  'Results/' + data_type + '_expt_' + str(expt_num) + '_col_combos.pkl'
joblib.dump(col_combos,f_name)

['Results/REM_expt_2_col_combos.pkl']